In [33]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics   #model optimization and valuation tools
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from utils import *

In [34]:
if os.path.exists('data_nyctaxi_2017_cleaned_2_model.csv'): # Check if the dataset is present on local disk and load it
    data = pd.read_csv('data_nyctaxi_2017_cleaned_2_model.csv')
else:
    print('Run Part-2 first')

In [34]:
data.head()

,Unnamed: 0,DOLocationID,PULocationID,RatecodeID,VendorID,extra,fare_amount,improvement_surcharge,month,mta_tax,...,trip_distance,Week,Week_day,Month_day,Hour,Trip_duration,Shift_type,Speed_mph,Tip_percentage,With_tip
0,0,13,234,1,1,0.5,16.0,0.3,March,0.5,...,3.70,3.0,1,13,21,18.550000,2.0,11.967655,16.626506,1
1,1,238,249,1,2,0.5,19.5,0.3,Nov,0.5,...,5.30,4.0,7,26,20,22.650000,2.0,14.039735,14.579055,1
2,2,238,142,1,2,0.5,5.5,0.3,Nov,0.5,...,0.93,5.0,3,29,19,5.300000,2.0,10.528302,20.000000,1
3,3,261,231,1,1,1.0,6.0,0.3,March,0.5,...,1.40,2.0,5,10,19,4.800000,2.0,17.500000,16.129032,1
4,4,236,162,1,2,0.5,6.5,0.3,Nov,0.5,...,1.71,5.0,3,29,0,4.416667,3.0,23.230189,13.812155,1


In [41]:
target = 'Tip_percentage'


var = ['DOLocationID_RATIO', 'PULocationID_RATIO', 'RatecodeID_RATIO', 'VendorID_RATIO', 'extra', 'fare_amount'
       , 'improvement_surcharge', 'month_RATIO', 'mta_tax', 'passenger_count', 'store_and_fwd_flag_RATIO'
       , 'tolls_amount', 'total_amount', 'Dropoff_dt_RATIO',  'Pickup_dt_RATIO', 'trip_distance', 'Week_RATIO'
       , 'Week_day_RATIO', 'Month_day_RATIO', 'Hour_RATIO', 'Trip_duration', 'Shift_type_RATIO', 'Speed_mph']

var_cat= ['RatecodeID', 'VendorID', 'month', 'Month_day','store_and_fwd_flag', 'Dropoff_dt',  'Pickup_dt', 'Week',
 'Week_day', 'Month_day', 'Hour', 'Shift_type','DOLocationID', 'PULocationID']

In [36]:
data["DOLocationID"] = data["DOLocationID"].astype('category')
data["PULocationID"] = data["PULocationID"].astype('category')
data["RatecodeID"] = data["RatecodeID"].astype('category')
data["VendorID"] = data["VendorID"].astype('category')
data["payment_type"] = data["payment_type"].astype('category')
data["Week"] = data["Week"].astype('category')
data["Week_day"] = data["Week_day"].astype('category')
data["Month_day"] = data["Month_day"].astype('category')
data["month"] = data["month"].astype('category')
data["Hour"] = data["Hour"].astype('category')
data["Dropoff_dt"]=data["Dropoff_dt"].astype('category')
data["Pickup_dt"]=data["Pickup_dt"].astype('category')
data["Shift_type"]=data["Shift_type"].astype('category')

### TRAIN TEST

In [37]:
mydata={}
mydata['train'], mydata['test']= mysplit(data, 0.6, 12345)

In [40]:
pd.set_option('display.max_columns', 500)
mydata['train'].head(2)

,Unnamed: 0,DOLocationID,PULocationID,RatecodeID,VendorID,extra,fare_amount,improvement_surcharge,month,mta_tax,passenger_count,payment_type,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,Dropoff_dt,Pickup_dt,trip_distance,Week,Week_day,Month_day,Hour,Trip_duration,Shift_type,Speed_mph,Tip_percentage,With_tip,rand,Tip_percentage_AUX,RatecodeID_RATIO,VendorID_RATIO,month_RATIO,Month_day_RATIO,store_and_fwd_flag_RATIO,Dropoff_dt_RATIO,Pickup_dt_RATIO,Week_RATIO,Week_day_RATIO,Month_day_RATIO_AUX,Hour_RATIO,Shift_type_RATIO,DOLocationID_RATIO,PULocationID_RATIO
0,1,238,249,1,2,0.5,19.5,0.3,Nov,0.5,1,1,N,3.55,0.0,24.35,2017-11-26 20:50:41,2017-11-26 20:28:02,5.30,4.0,7,26,20,22.65,2.0,14.039735,14.579055,1,0.316376,14.579055,16.121914,16.240243,16.120567,16.068755,16.110197,16.109908,16.109908,16.111109,16.042117,16.068755,16.038011,16.063614,16.084693,16.001696
1,2,238,142,1,2,0.5,5.5,0.3,Nov,0.5,1,1,N,1.70,0.0,8.50,2017-11-29 20:05:08,2017-11-29 19:59:50,0.93,5.0,3,29,19,5.30,2.0,10.528302,20.000000,1,0.183919,20.000000,16.121914,16.240243,16.120567,16.161541,16.110197,16.109908,16.109908,16.159602,16.125081,16.161541,15.956789,16.063614,16.084693,16.193381


In [50]:
var = ['DOLocationID', 'PULocationID', 'RatecodeID', 'VendorID', 'extra', 'fare_amount'
       , 'improvement_surcharge', 'month', 'mta_tax', 'passenger_count', 'store_and_fwd_flag'
       , 'tolls_amount', 'total_amount', 'trip_distance', 'Week'
       , 'Week_day', 'Month_day', 'Hour', 'Trip_duration', 'Shift_type', 'Speed_mph']

var_cat= ['month']
# 'RatecodeID', 'VendorID', 'month', 'Month_day','store_and_fwd_flag', 'Dropoff_dt',  'Pickup_dt', 'Week',
# 'Week_day', 'Month_day', 'Hour', 'Shift_type','DOLocationID', 'PULocationID']

In [ ]:
mydata['train'] = encode_vars_ratio(mydata, mydata['train'], mydata['train'], var_cat, target, True, 100)
mydata['test'] = encode_vars_ratio(mydata, mydata['test'], mydata['test'], var_cat, target, True, 100)

In [22]:
regressor = DecisionTreeRegressor(max_features='sqrt', splitter='best', min_samples_split=5, min_samples_leaf=10, max_depth=10)
regressor.fit(mydata['train'][var], mydata['train'][target])
# Apply model to test
mydata['test'][target] = regressor.predict(mydata['test'][var])  

In [28]:
scores_dc = cross_val_score(regressor,mydata['test'][var], mydata['test'][target], scoring='mean_absolute_error', cv=2)
print('crosvalidation mean absolute error',scores_dc.mean())

importances=regressor.feature_importances_
print(var)
print(importances)


crosvalidation mean absolute error -0.6120749739989637
['DOLocationID_RATIO', 'PULocationID_RATIO', 'RatecodeID_RATIO', 'VendorID_RATIO', 'extra', 'fare_amount', 'improvement_surcharge', 'month_RATIO', 'mta_tax', 'passenger_count', 'store_and_fwd_flag_RATIO', 'tolls_amount', 'total_amount', 'Dropoff_dt_RATIO', 'Pickup_dt_RATIO', 'trip_distance', 'Week_RATIO', 'Week_day_RATIO', 'Month_day_RATIO', 'Hour_RATIO', 'Trip_duration', 'Shift_type_RATIO', 'Speed_mph', 'month_RATIO']
[0.01761764 0.01640386 0.02158038 0.00301351 0.0316825  0.28791411
 0.         0.00074903 0.00524059 0.00121142 0.         0.011674
 0.31032999 0.         0.         0.05729852 0.00078909 0.00117068
 0.00328993 0.00926072 0.17790456 0.01662218 0.02573772 0.00050957]


In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

In [51]:
tic = dt.datetime.now()
from sklearn.ensemble import RandomForestRegressor
# optimize n_estimator through grid search
def optimize_num_trees(alg,param_test,scoring_method,train,predictors,target):
    """
    This functions is used to tune paremeters of a predictive algorithm
    alg: sklearn model,
    param_test: dict, parameters to be tuned
    scoring_method: str, method to be used by the cross-validation to valuate the model
    train: pandas.DataFrame, training data
    predictors: list, labels to be used in the model training process. They should be in the column names of dtrain
    target: str, target variable
    """
    gsearch = GridSearchCV(estimator=alg, param_grid = param_test, scoring=scoring_method,n_jobs=2,iid=False,cv=5)
    gsearch.fit(train[predictors],train[target])
    return gsearch

# optimize n_estimator through grid search
param_test = {'n_estimators':range(50,200,25)} # define range over which number of trees is to be optimized
# initiate classification model
#rfr = RandomForestRegressor(min_samples_split=2,max_depth=5,max_features='auto',random_state = 10)
rfr = RandomForestRegressor()#n_estimators=100)
# get results of the search grid
gs_rfr = optimize_num_trees(rfr,param_test,'mean_squared_error',mydata['train'],var,target)

KeyboardInterrupt: 

In [ ]:
# plot optimization results
def plot_opt_results(alg):
    cv_results = []
    for i in range(len(param_test['n_estimators'])):
        cv_results.append((alg.grid_scores_[i][1],alg.grid_scores_[i][0]['n_estimators']))
    cv_results = pd.DataFrame(cv_results)
    plt.plot(cv_results[1],cv_results[0])
    plt.xlabel('# trees')
    plt.ylabel('score')
    plt.title('optimization report')

In [ ]:
print gs_rfr.grid_scores_, gs_rfr.best_params_, gs_rfr.best_score_

# plot optimization results
#plot_opt_results(gs_rfr)

# cross validate the best model with optimized number of estimators
modelfit(gs_rfr.best_estimator_,train,predictors,target,'mean_squared_error')

ypred = gs_rfr.best_estimator_.predict(test[predictors])

print 'RFR test mse:',metrics.mean_squared_error(ypred,test.Tip_percentage)
print 'RFR r2:', metrics.r2_score(ypred,test.Tip_percentage)
print dt.datetime.now()-tic
plot_opt_results(gs_rfr)